In [1]:
## Import der Module
# Pyhton             Version 3.11.1
# Selenium           Version 4.9.1
# webdriver_manager  Version 3.8.6
# pandas             Version  2.0.1
# datetime           Version 5.1
# sqlalchemy         Version 2.0.13
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import datetime
from sqlalchemy import create_engine
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
#---------------------------------------------------------------------------
## Hilfsfunktionen
# Log File
def schreibe_log_file(log_eintrag):
    datei='log-Indeed.txt' # Name der Log-Datei
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Überprüfen, ob die Log-Datei bereits existiert
    if not os.path.exists(datei):
        # Wenn die Log-Datei nicht existiert, wird sie erstellt und der Log-Eintrag wird hineingeschrieben
        with open(datei, 'w') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
    else:
        # Wenn die Log-Datei bereits existiert, wird der Log-Eintrag an das Ende der Datei angehängt
        with open(datei, 'a') as file:
            file.write(f"{timestamp}, {log_eintrag}" + '\n')
# E-Mail - Benachrichtigung bei Fehlern
def schreibe_e_mail(message, Subject="fehlerhaft"):
    # SMTP-Server-Konfiguration
    host = "smtp.web.de"###imap.web.de
    port = 587
    log_in_id = "dc-jobs"
    passwort = "DatacraftKurs0822!"
    # Erstellen der E-Mail-Nachricht
    msg = MIMEMultipart()
    msg['From'] = log_in_id + "@web.de" # Absender
    msg['To'] = "jobs@data-craft.de" # Empfänger
    msg['Subject'] = f'Indeed Abfrage {Subject}' # Betreff der E-Mail
    msg.attach(MIMEText(message, 'plain')) # Hinzufügen des Nachrichtentextes zur E-Mail
    # Verbindung zum SMTP-Server herstellen und E-Mail senden
    with smtplib.SMTP(host=host, port=port) as mail:
        mail.starttls() # Starte TLS-Verschlüsselung
        mail.login(log_in_id, passwort) # Anmeldung am SMTP-Server
        mail.send_message(msg) # E-Mail senden
# random Wartezeit zum für seitenaufbau oder so
def wartezeit(zeit=3):
    """
    Diese Funktion fügt eine zufällige Wartezeit hinzu, bevor sie fortgesetzt wird.
    :param zeit: Die maximale Wartezeit in Sekunden (Standardwert ist 3).
    :return: None
    """
    time.sleep(random.randint(1, zeit))
#-------------------------------------------------------------------------------
## SQL-Server verbindung erstellen
# SQL-Server-Konfiguration
#host="datacraft-db.cf3hyz1fwdiw.eu-central-1.rds.amazonaws.com"
host='datacraft-jobs.postgres.database.azure.com'
database="postgres"
user="jobs_rw"
password="01QT3X48xCi4KXUckxDq"
tabelle_Rohdaten="jobs.rohdaten"
# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')
#-------------------------------------------------------------------------------
## Der Scraper
def scraper(jobtitel, suchort='Deutschland'):
    '''
    Der eigentliche Scraper für die Seite
   
    :param jobtitel: Der Titel des Jobs, nach dem gesucht werden soll.
    :param suchort: Der Ort, an dem nach Jobs gesucht werden soll. Standardwert ist 'Deutschland'.
    '''
   
    schreibe_log_file(f'suche nach {jobtitel}')
    # ChromeOptions erstellen
    chrome_options = webdriver.ChromeOptions()
    #Headless-Modus aktivieren
    #chrome_options.add_argument("--headless")##### hier war auskommentiert hier ohne zu sehen der webseite alles passiert im hintergrund
    # Browserfenster öffnen nach Einstellung in den Optionen
    try:
        #driver_path = ChromeDriverManager().install()  #Neu: Chromedriver herunterladen und den Pfad
        #service = ChromeService(executable_path=driver_path)  # Neu: ChromeService mit dem Pfad erstellen
        #driver = webdriver.Chrome(service=service, options=chrome_options)  # Neu: Initialisiere den Chromedriver mit den Optionen
        #schreibe_log_file('Browser geoeffnet')# neu eingesetzt kann aber raus
        #driver_path = ChromeDriverManager().install()  #Neu: Chromedriver herunterladen und den Pfad
        driver = webdriver.Chrome(options=chrome_options)  # Neu: Initialisiere den Chromedriver mit den Optionen
        #driver.service.path = driver_path  # Neu: Setze den Pfad des Dienstes auf den heruntergeladenen #Chromedriver
        schreibe_log_file('Browser geoeffnet')
        # das war alt
        #driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), #options=chrome_options)
        #schreibe_log_file('Browser geoeffnet')
    except:
        schreibe_log_file('Browser konnte nicht geoeffnet werden')
        schreibe_e_mail(f'Browser konnte nicht geoeffnet werden: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')
    # Webseite aufrufen
    wartezeit(3)
    driver.get("https://www.indeed.de/")
    # Fenster Maximieren
    driver.maximize_window() # evtl. wenn Headless-Modus aktiv ist auskommentieren  ### habe auskommentiert schau ob klappt
    #------------------------------------------------------------------------------
    ## Cookies akzeptieren
    wartezeit(3)
    try:
        driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass
    wartezeit(3)
    #------------------------------------------------------------------------------
    ## Suchfelder finden, anwählen und befüllen
    # Suchfeld Jobbezeichnung -> finden und befüllen
    try:
        suchfeld_jobtitel=driver.find_element(By.XPATH,'//*[@id="text-input-what"]')
        suchfeld_jobtitel.send_keys(jobtitel)
        wartezeit(3)
    # Suchfeld Ort -> finden und befüllen
        suchfeld_ort=driver.find_element(By.XPATH,'//*[@id="text-input-where"]')
        suchfeld_ort.send_keys(suchort)
        # Anfrage mit Enter/Return abschicken
        suchfeld_ort.send_keys(Keys.RETURN)
    except:
        schreibe_log_file(f'Suchdaten eingeben nicht möglich: Jobtitel:-{jobtitel}; Suchort-{suchort}')
        print(f'Suchdaten eingeben nicht möglich: Jobtitel:-{jobtitel}; Suchort-{suchort}')
        return
    wartezeit(3)
    #-----------------------------------------------------------------------------
    ## nach Datum sortieren
    driver.find_element(By.XPATH, '//*[@id="jobsearch-JapanPage"]/div/div/div[5]/div[1]/div[4]/div/div/div[1]/span[2]/a').click()
    wartezeit(3)
    #-----------------------------------------------------------------------------
    ## einblendung entfernen
    try:
        driver.find_element(By.XPATH, '//*[@id="mosaic-desktopserpjapopup"]/div[1]/button').click()
    except:
        pass
    wartezeit(3)
    #-----------------------------------------------------------------------------
    link_liste_scraped=[]
    ## link_liste_scraped befüllen von erster Seite
    anzeigen=driver.find_elements(By.CLASS_NAME, 'jcs-JobTitle')
    for anzeige in anzeigen:
        link_liste_scraped.append(anzeige.get_attribute('href'))
    # auf "nächste Seite" Button klicken
    try:
        driver.find_element(By.XPATH , '//*[@id="jobsearch-JapanPage"]/div/div/div[5]/div[1]/nav/div[6]/a').click()
    except:
        pass
    #--------------------------------------------------------------------------------
    ## von weiter Seiten die Links holen
    for i in range(9):
        anzeigen=driver.find_elements(By.CLASS_NAME, 'jcs-JobTitle')
        for anzeige in anzeigen:
            link_liste_scraped.append(anzeige.get_attribute('href'))
        # auf "nächste Seite" Button klicken
        try:
            driver.find_element(By.XPATH , '//*[@id="jobsearch-JapanPage"]/div/div/div[5]/div[1]/nav/div[7]/a').click()
        except:
            pass
        wartezeit()
    #--------------------------------------------------------------------------------
    # überprüfen ob link_liste_scraped leer ist da evtl. die Class name geändert wurde
    if len(link_liste_scraped) == 0:
        schreibe_log_file('Keine Links auf gefunden')
        schreibe_e_mail('Keine Links auf Webseite gefunden:\n\n Hinweis: Classenname überprüfen für alle Stellenanzeigen!')
        return
    else:
        pass
    #--------------------------------------------------------------------------------
    # Duplikate entfernen
    link_liste_scraped_ohne_duplikate = list(set(link_liste_scraped))
    schreibe_log_file(f'Es wurden {len(link_liste_scraped)-len(link_liste_scraped_ohne_duplikate)} Duplikate in der "Link Liste" entfernt')
    schreibe_log_file("link liste erstellt")
    #--------------------------------------------------------------------------------
    # Lese die bestehenden URLs aus der Datenbank
    existing_urls = pd.read_sql_query(f"SELECT url FROM {tabelle_Rohdaten}", connection)
    schreibe_log_file(f'{len(existing_urls)} in der datenbak bereits vorhanden')
    # Filtere den DataFrame, um nur neue URLs zu behalten
    link_liste = [url for url in link_liste_scraped_ohne_duplikate if url not in existing_urls['url'].values]
    #--------------------------------------------------------------------------------
    seiten_inhalt_html_liste = []
    seiten_inhalt_liste = []
    URL_Liste = []
    Datum_Liste = []
    try:
        schreibe_log_file('Abfrage begonnen')
        for link in link_liste:
            # link öffnen
            driver.get(link)
            wartezeit()
            # scrapen der daten und befüllen der Listen
            seiten_inhalt_html_liste.append(driver.find_element(By.CLASS_NAME, 'jobsearch-JobComponent').get_attribute('innerHTML'))
            seiten_inhalt_liste.append(driver.find_element(By.CLASS_NAME, 'jobsearch-JobComponent').text)
            URL_Liste.append(link)
            Datum_Liste.append(datetime.datetime.now())
            schreibe_log_file(f'Daten wurden gezogen: {link}')
            wartezeit(5)
    except:
        schreibe_log_file("Abfrage abgebrochen !!!")
        schreibe_e_mail(f"Die Abfrage wurde abgebrochen. -- {datetime.datetime.now()} --")
        schreibe_log_file('Abfrage beendet')
    #----------------------------------------------------------------------------------
    # Schließen des Browsers
    wartezeit(1)
    driver.quit()
    schreibe_log_file('Browser geschlossen')
    #----------------------------------------------------------------------------------
    # überprüfen ob die Listen gleich lang sind und den DataFrame erstellen
    if len(set(map(len, [seiten_inhalt_html_liste,
                        seiten_inhalt_liste,
                        URL_Liste,
                        Datum_Liste]))) > 1:
        schreibe_log_file('Listen sind verschieden lang')
        schreibe_e_mail('Listen sind nicht gleich lang')
        return
    else:
        # DataFrame erstellen
        df = pd.DataFrame({"seite": "indeed",
                        "seiten_inhalt_html": seiten_inhalt_html_liste,
                        "seiten_inhalt": seiten_inhalt_liste,
                        "url": URL_Liste,
                        "datum":Datum_Liste,
                        "storno": False})
    #-----------------------------------------------------------------------------------
    ## Daten in die Datenbank einfügen
    # Schreibe den bereinigten DataFrame in die Datenbank
    df.to_sql(name=tabelle_Rohdaten.split('.')[1],
            schema=tabelle_Rohdaten.split('.')[0],
            con=connection, if_exists='append', index=False, )
    schreibe_log_file(f'Es wurden {len(df)} Daten von Indeed hinzugefügt')
    # Verbindung zur SQL-Datenbank schließen
    connection.dispose()
print("fertig")

fertig


In [2]:
scraper("Data Analyst")

In [3]:
# Ich arbeite mit zwei Computern, und leider hat die Datei für die Suchbegriffe (Jobtitel) jeweils ein anderen Speicherort
try:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\Documents\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
      sheet_name='Indeed')
except:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\Documents\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
     sheet_name='Indeed')  
display(suchbegriffe)

,Jobtitel
0,Customer Data Analyst
1,Data Analyst
2,Junior Data Analyst
3,(Junior) Data Analyst
4,Datenanalyst
5,Data Engineer / Digital Analyst
6,BI Analyst* / Data Analyst*
7,Datenbankentwickler / -analyst
8,Data Analytics & Visualization Analyst
9,BI Data Analyst


In [4]:
print(f'{len(suchbegriffe)} Jobtitel gefunden')
for i, jobtitel in enumerate(suchbegriffe.Jobtitel):
    print(f'{i}: Suche nach "{jobtitel}", Start: {datetime.datetime.now().strftime("%H:%M:%S")} Uhr')
    scraper(jobtitel)
print("")
print("fertig")

12 Jobtitel gefunden
0: Suche nach "Customer Data Analyst", Start: 10:36:28 Uhr
1: Suche nach "Data Analyst", Start: 10:40:04 Uhr
2: Suche nach "Junior Data Analyst", Start: 10:42:03 Uhr


UnboundLocalError: cannot access local variable 'driver' where it is not associated with a value

In [ ]:
pip list